<a href="https://colab.research.google.com/github/maheshwarampranay/Text-Summarizer/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstractive Text Summarizer - With Fine tuning the BART Transformer

In [1]:
!pip install -U datasets

Let us use this dialogsum dataset to train our model !! We will be using the BART pre-trained transformer from the hugging face.

In [2]:
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Let us now, inspect the dataset.

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Performing Exploratory Analysis over the data !!

In [4]:
ds['train'][1]['dialogue']

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [5]:
ds['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

#Fine-Tuning the model


In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

#Tokenization

In [7]:
#tokenisation
def preprocess_function(batch):
    """returns
    input_ids - which are tokenized format of your inputs - vectorised format
    attention mask - will determine which token should be attended and which to be removed - tokens with padding are ignored - a binary variable
    labels"""
    source = batch['dialogue']
    target = batch['summary']
    source_ids = tokenizer(source, truncation=True, padding='max_length', max_length=128)
    target_ids = tokenizer(target, truncation=True, padding='max_length', max_length=128)

    labels = target_ids['input_ids']
    labels = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels]

    return {
        'input_ids': source_ids['input_ids'],
        'attention_mask': source_ids['attention_mask'],
        'labels': labels
    }

mapping the function to our dataset to our function

In [8]:
df_source = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [9]:
#Training arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content",
    per_device_train_batch_size = 8,#Hyperparameter
    num_train_epochs = 3, #Hyperparameter
    remove_unused_columns=True  # True beacuse we need only required colums -- like the dialogue and summary
)

In [10]:
#creating trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_source['train'],
    eval_dataset=df_source['test']
)

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maheshwarampranaychary (maheshwarampranaychary-chaitanya-bharathi-institute-of-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.595700
1000,1.500100
1500,1.451800
2000,1.102700
2500,1.050100
3000,1.041900
3500,0.813700
4000,0.736500
4500,0.717300


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=4674, training_loss=1.097705681098543, metrics={'train_runtime': 6000.3157, 'train_samples_per_second': 6.23, 'train_steps_per_second': 0.779, 'total_flos': 1.012579625336832e+16, 'train_loss': 1.097705681098543, 'epoch': 3.0})

In [12]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)

{'eval_loss': 1.8645567893981934, 'eval_runtime': 50.3796, 'eval_samples_per_second': 29.774, 'eval_steps_per_second': 3.732, 'epoch': 3.0}


In [13]:
# Save the model and tokenizer after training
model.save_pretrained("/content/model_saved")
tokenizer.save_pretrained("/content/model_saved")

('/content/model_saved/tokenizer_config.json',
 '/content/model_saved/special_tokens_map.json',
 '/content/model_saved/vocab.json',
 '/content/model_saved/merges.txt',
 '/content/model_saved/added_tokens.json',
 '/content/model_saved/tokenizer.json')

This model which we have tuned, and trained - we will be using as our model to perform summarizations.

We have saved this model successfully. We will be using this in our python script - integrating with UI and necessary logic to perform text summarization